# Project 1


In [245]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('./Data/young-people-survey-responses.csv')
print(data.head(5))

In [ ]:
print(data.describe())
print("\n median music", data['Music'].median())
print("\n median techno", data['Techno'].median())


In [ ]:
print(data.isna().sum())

In [ ]:
sns.heatmap(data.isna())

## Drop Variables that are not necessary

In [250]:
# we can drop loneliness because that is the variable we are trying to guess

y = data["Loneliness"]
data.drop('Loneliness', axis=1, inplace=True )

## Missing Values


In [ ]:
percent_missing_vals = data.agg(lambda x: x.isna().mean() * 100)

print(percent_missing_vals)
# None are over 10 percent

In [252]:
# fewer than 10%, fill with min, max, mean or median.
def fill_avg(x,col):
    x[col].replace(np.nan, x[col].mean(), inplace=True)
    return x


def fill_mode(x, col):
    x[col].replace(np.nan, x[col].mode(), inplace=True)
    
def fill_median(x, col):
    x[col].replace(np.nan, x[col].median(), inplace=True)



In [ ]:
# fill with median since they are ordinal values.
# we don't want 2.45 on a scale of 1-5
fill_median(data, col='Techno')
fill_median(data, col='Movies')
fill_median(data, col='Music')
fill_median(data, col="History")
fill_median(data, col='Mathematics')
fill_median(data, col='Pets')
fill_median(data, col='Spiders')
fill_median(data, col="Siblings")
fill_median(data, col="Parents' advice")
fill_median(data, col="Finances")



fill_avg(data, col="Age")

data['Gender'].replace(np.nan, data['Gender'].mode()[0], inplace=True)
data['Village - town'].replace(np.nan, data['Village - town'].mode()[0], inplace=True)

sns.heatmap(data.isna())



In [ ]:
print(data.isna().sum())

In [ ]:
display(data)

## Encode Categorical Variables

In [256]:
# categorical variables: 'Gender', 'Internet Usage', 'village - town'

data = data.join(pd.get_dummies(data['Gender'], dtype=int)).drop('Gender', axis=1)
data = data.join(pd.get_dummies(data['Village - town'], dtype=int)).drop('Village - town', axis=1)


In [ ]:

display(data)

In [ ]:
print(data['Internet usage'].unique())

In [259]:
# map Internet usage from low to high
def ordinal_encode(x,col):
    order = {
        'no time at all':0,
        np.nan:0,
        'less than an hour a day':1,
        'few hours a day':2,
        'most of the day':3
    }
    x[col] = x[col].map(order)
    return x

In [260]:
data = ordinal_encode(data, col="Internet usage")

In [263]:
display(data)

,Unnamed: 0,Music,Techno,Movies,History,Mathematics,Pets,Spiders,Parents' advice,Internet usage,Finances,Age,Siblings,female,male,city,village
0,0,5.0,1.0,5.0,1.0,3.0,4.0,1.0,4.0,2,3.0,20.0,1.0,1,0,0,1
1,1,4.0,1.0,5.0,1.0,5.0,5.0,1.0,2.0,2,3.0,19.0,2.0,1,0,1,0
2,2,5.0,1.0,5.0,1.0,5.0,5.0,1.0,3.0,2,2.0,20.0,2.0,1,0,1,0
3,3,5.0,2.0,5.0,4.0,4.0,1.0,5.0,2.0,3,2.0,22.0,1.0,1,0,1,0
4,4,5.0,2.0,5.0,3.0,2.0,1.0,1.0,3.0,2,4.0,20.0,1.0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1005,5.0,3.0,5.0,4.0,1.0,4.0,2.0,4.0,2,3.0,20.0,1.0,1,0,1,0
1006,1006,4.0,4.0,5.0,4.0,5.0,5.0,1.0,4.0,1,3.0,27.0,5.0,0,1,0,1
1007,1007,4.0,1.0,4.0,2.0,3.0,5.0,2.0,4.0,3,1.0,18.0,0.0,1,0,1,0
1008,1008,5.0,2.0,5.0,3.0,1.0,4.0,3.0,3.0,3,3.0,25.0,1.0,1,0,1,0


## Divide into Training and Testing Groups

In [264]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=1)


## Scaling

Because most of them are on a scale of 1-5, I will standardize them to be between -1 and 1

In [266]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

## Cross Validation